In [ ]:
#Carregar Bibliotecas

#pip install
from keras.layers import Input, Dense
from keras.models import Model
from keras import regularizers
from keras.datasets import mnist
from keras.models import Sequential
#import GraphViz
import keras
from keras.layers import Conv2D, MaxPooling2D, Input, Dense, Flatten
from keras.models import Model
from keras.utils.vis_utils import plot_model
import numpy as np
import tensorflow as tf
import os
import cv2
from keras.layers import Input,Dense,Flatten,Dropout,merge,Reshape,Conv2D,MaxPooling2D,UpSampling2D,Conv2DTranspose
from keras.layers.normalization import BatchNormalization
from keras.models import Model,Sequential
from keras.callbacks import ModelCheckpoint
from keras.optimizers import Adadelta, RMSprop,SGD,Adam
from keras import regularizers
from keras import backend as K
import skimage.transform
import matplotlib.pyplot as plt
import matplotlib
matplotlib.interactive(True)



#Carregar Dados

x_train = np.load('c:/Users/Natan/Documents/Universidade/3_TCC_Eng_Biomed/TCC_Novos/TCC_Experimentos/IXI_T1_train.npy')
np.shape(x_train)
    
x_test = np.load('c:/Users/Natan/Documents/Universidade/3_TCC_Eng_Biomed/TCC_Novos/TCC_Experimentos/IXI_T1_valid.npy')
np.shape(x_test)

x_train = np.reshape(x_train, (len(x_train), 119, 119))  # adapt this if using `channels_first` image data format
np.shape(x_train)

x_test = np.reshape(x_test, (len(x_test), 119, 119))  # adapt this if using `channels_first` image data format
np.shape(x_test)

x_train = skimage.transform.resize(x_train,(600,60,60,1))
np.shape(x_train)

x_test = skimage.transform.resize(x_test,(88,60,60,1))
np.shape(x_test)


x_train = np.reshape(x_train, (len(x_train), 60, 60, 1))  # adapt this if using `channels_first` image data format
np.shape(x_train)

x_test = np.reshape(x_test, (len(x_test), 60, 60, 1))  # adapt this if using `channels_first` image data format
np.shape(x_test)


batch_size = 128
aux_epoch = 100
epochs = aux_epoch
inChannel = 1
lin, col = 60, 60


#input_img = Input(shape = (x, y, inChannel))

input_img = Input(shape=(lin, col, 1))

#conv1 
x = Conv2D(32, (3, 3), activation='relu', padding='same')(input_img) #28 x 28 x 32
np.shape(x)

#conv1 
x = BatchNormalization()(x)
np.shape(x)

#conv1 
x = Conv2D(32, (3, 3), activation='relu', padding='same')(x)
np.shape(x)

#conv1
x = BatchNormalization()(x)
np.shape(x)

#pool1 
x = MaxPooling2D(pool_size=(2, 2))(x) #14 x 14 x 32
np.shape(x)

#conv2 
x = Conv2D(64, (3, 3), activation='relu', padding='same')(x) #14 x 14 x 64
np.shape(x)

#conv2 
x = BatchNormalization()(x)
np.shape(x)

#conv2 
x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
np.shape(x)

#conv2 
x = BatchNormalization()(x)
np.shape(x)

#pool2 
x = MaxPooling2D(pool_size=(2, 2))(x) #7 x 7 x 64
np.shape(x)

#conv3 
x = Conv2D(128, (3, 3), activation='relu', padding='same')(x) #7 x 7 x 128 (small and thick)
np.shape(x)

#conv3 
x = BatchNormalization()(x)
np.shape(x)

#conv3 
x = Conv2D(128, (3, 3), activation='relu', padding='same')(x)
np.shape(x)
#conv3 
encoded = BatchNormalization()(x)
np.shape(encoded)


#decoder
#conv4 
x = Conv2D(64, (3, 3), activation='relu', padding='same')(encoded) #7 x 7 x 128
np.shape(x)

#conv4 
x = BatchNormalization()(x)
np.shape(x)

#conv4 
x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
np.shape(x)

#conv4 
x = BatchNormalization()(x)
np.shape(x)

#up1 
x = UpSampling2D((2,2))(x) # 14 x 14 x 128
np.shape(x)

#conv5 
x = Conv2D(32, (3, 3), activation='relu', padding='same')(x) # 14 x 14 x 64
np.shape(x)

#conv5 
x = BatchNormalization()(x)
np.shape(x)

#conv5 
x = Conv2D(32, (3, 3), activation='relu', padding='same')(x)
np.shape(x)

#conv5 
x = BatchNormalization()(x)
np.shape(x)

#up2 
x = UpSampling2D((2,2))(x) # 28 x 28 x 64
np.shape(x)

#decoded 
decoded = Conv2D(1, (2, 2), activation='sigmoid', padding='same')(x) # 28 x 28 x 1
np.shape(decoded)


encoder = Model(input_img, encoded)
autoencoder = Model(input_img, decoded)
autoencoder.compile(loss='mean_squared_error', optimizer = RMSprop())


autoencoder.summary()




In [ ]:
from keras.utils import plot_model
plot_model(autoencoder,to_file='ORIGINAL_TCC.png',show_shapes=True)

In [ ]:
autoencoder_train=autoencoder.fit(x_train, x_train, batch_size=batch_size,epochs=epochs,verbose=1,
                                    validation_data=(x_test, x_test))

In [ ]:
loss = autoencoder_train.history['loss']
val_loss = autoencoder_train.history['val_loss']
epochs = range(aux_epoch)
plt.figure()
plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()
plt.show()

In [ ]:
pred = autoencoder.predict(x_test)
a = 8 # Anda de 4 em 4 por paciente
n = 4  
plt.figure()
for i in range(0,n):
    ax = plt.subplot(2, n, i + 1)
    #Test is input
    plt.imshow(x_test[a+i].reshape(lin,col))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
    ax = plt.subplot(2, n, i + 1 + n)
    plt.imshow(pred[a+i].reshape(lin , col))
    #decoded_imgs
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
#
#
#

plt.show()
